In [29]:
from dotenv import load_dotenv

load_dotenv()

True

## Propm template 🧐

In [7]:
from langchain.prompts import PromptTemplate

propmt = PromptTemplate.from_template("Describe an object that you find {adjective} and why it has effect on you.")

In [8]:
propmt.format(adjective="awful")

'Describe an object that you find awful and why it has effect on you.'

In [10]:
template = "you are useful assistent that translates from {language_input} to {language_output} of text: {text}"
template

'you are useful assistent that translates from {language_input} to {language_output} of text: {text}'

In [11]:
text = "I love programming"

In [19]:
prompt_template = PromptTemplate(
    input_variables=["language_input", "language_output", "text"],
    template=template,
)

## Wrapper Wikipedia 🌍

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [24]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [15]:
print(tool.name)
print(tool.description)
print(tool.args)

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
{'query': {'title': 'Query', 'description': 'query to look up on wikipedia', 'type': 'string'}}


In [25]:
tool.run({ 'query': "what is the purpose of cryptocurrencies" })

'Page: Cryptocurrency\nSummary: A cryptocurrency, crypto-currency, or crypto is a digital currency designed to work as a medium of exchange through a computer network that is not reliant on any central authority, such as a government or bank, to uphold or maintain it. \nIndividual coin ownership records are stored in a digital ledger, which is a computerized database using strong cryptography to secure transaction records, control the creation of additional coins, and verify the transfer of coin ow'

## Tavily Search 🔍

In [30]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
search.invoke({ 'query': "what is the purpose of cryptocurrencies" })

[{'url': 'https://www.investopedia.com/terms/c/cryptocurrency.asp',
  'content': "Removes single points of failure\nEasier to transfer funds between parties\nRemoves third parties\nCan be used to generate returns\nRemittances are streamlined\nTransactions are pseudonymous\nPseudonymity allows for criminal uses\nHave become highly centralized\nExpensive to participate in a network and earn\nOff-chain security issues\nPrices are very volatile\nAdvantages Explained\nCryptocurrencies represent a new, decentralized paradigm for money. Thus, a system with cryptocurrencies eliminates the possibility of a single point of failure—such as a large financial institution setting off a cascade of global crises, such as the one triggered in 2008 by the failure of large investment banks in the U.S.\nCryptocurrencies promise to make transferring funds directly between two parties easier without needing a trusted third party like a bank or a credit card company. Here are some of the types you'll find wi

## Calling LLM 📞

In [13]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

In [20]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
answer = chain.invoke(
    input={
        "language_input": "espanish",
        "language_output": "english",
        "text": text
    }
)

print(answer)